In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


This will likely need to change everytime if your path looks different to mine... thats annoying

In [36]:
PATH_TO_PROJECT = '/drive/Othercomputers/My MacBook Pro/FAERS-analytics/'

In [37]:
%cd $PATH_TO_PROJECT

[Errno 2] No such file or directory: '/drive/Othercomputers/My MacBook Pro/FAERS-analytics/'
/content/drive/Othercomputers/My MacBook Pro/FAERS-analytics


In [38]:
!ls

 data   data-out  'FAERS analytics.ipynb'   README.md


In [39]:
import os

output_path_dir = 'data-out/'
input_path_dir = 'data/'

if not os.path.isdir(output_path_dir):
  !mkdir $output_path_dir

for file in os.listdir(input_path_dir):
  if file.endswith(".zip"):
    input_path_file = input_path_dir + file
    output_path_file = output_path_dir + file.split('.zip')[0]
    if not os.path.isdir(output_path_file):
      !mkdir $output_path_file
      !unzip $input_path_file -d $output_path_file

In [40]:
!ls $output_path_dir

FAERS_ASCII_2023_Q3  FAERS_ASCII_2023_Q4


In [41]:
q3_dir = output_path_dir + 'FAERS_ASCII_2023_Q3/ASCII/'
!ls $q3_dir

ASC_NTS.pdf   DRUG23Q3.pdf  INDI23Q3.txt  REAC23Q3.pdf	RPSR23Q3.txt
DEMO23Q3.pdf  DRUG23Q3.txt  OUTC23Q3.pdf  REAC23Q3.txt	THER23Q3.pdf
DEMO23Q3.txt  INDI23Q3.pdf  OUTC23Q3.txt  RPSR23Q3.pdf	THER23Q3.txt


In [48]:
q4_dir = output_path_dir + 'FAERS_ASCII_2023_Q4/ASCII/'
!ls $q4_dir

ASC_NTS.pdf   DRUG23Q4.pdf  INDI23Q4.txt  REAC23Q4.pdf	RPSR23Q4.txt
DEMO23Q4.pdf  DRUG23Q4.txt  OUTC23Q4.pdf  REAC23Q4.txt	THER23Q4.pdf
DEMO23Q4.txt  INDI23Q4.pdf  OUTC23Q4.txt  RPSR23Q4.pdf	THER23Q4.txt


In [47]:
import pandas as pd

data = {}
directories = [q3_dir, q4_dir]


for directory in directories:
  for file in os.listdir(directory):
    if file.endswith(".txt"):
      input_path_file = './' + directory + file
      output_data_frame_name = file.split('.txt')[0]
      output_data_frame = pd.read_csv(input_path_file, sep='$')
      data[output_data_frame_name] = output_data_frame

data.keys()


<ipython-input-47-bd5f6272dbb1>:12: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  output_data_frame = pd.read_csv(input_path_file, sep='$')
<ipython-input-47-bd5f6272dbb1>:12: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  output_data_frame = pd.read_csv(input_path_file, sep='$')


dict_keys(['DEMO23Q3', 'DRUG23Q3', 'INDI23Q3', 'OUTC23Q3', 'REAC23Q3', 'RPSR23Q3', 'THER23Q3', 'DEMO23Q4', 'DRUG23Q4', 'INDI23Q4', 'OUTC23Q4', 'REAC23Q4', 'RPSR23Q4', 'THER23Q4'])

In [57]:
for df_name, df in data.items():
    print('Current Table Name: ' + df_name)
    print('Current Table Shape: ' + str(df.shape))
    print('Current Table Snippet: \n')
    print(df.head())
    print ('\n\n -------------------------------------------------------------------------- \n\n')

Current Table Name: DEMO23Q3
Current Table Shape: (407522, 25)
Current Table Snippet: 

    primaryid    caseid  caseversion i_f_code    event_dt      mfr_dt  \
0   100051542  10005154            2        F  20230718.0  20230718.0   
1  1001678123  10016781           23        F  20120330.0  20230905.0   
2   100277024  10027702            4        F  20120101.0  20230808.0   
3   100503669  10050366            9        F  20140310.0  20230624.0   
4   100762232  10076223            2        F         NaN  20230925.0   

   init_fda_dt    fda_dt rept_cod auth_num  ... age_grp  sex e_sub    wt  \
0     20140312  20230726      PER      NaN  ...     NaN  NaN     Y   NaN   
1     20140318  20230912      EXP      NaN  ...     NaN    F     Y   NaN   
2     20140321  20230815      EXP      NaN  ...       E    M     Y  47.0   
3     20140401  20230808      EXP      NaN  ...       A    M     Y  98.0   
4     20140414  20230929      EXP      NaN  ...     NaN    F     Y   NaN   

  wt_cod   rept_